In [1]:
import meshio
import pygmsh
import pygalmesh
import numpy as np
import copy
from mshr import *
from dolfin import *
from collections import Counter
import matplotlib.pyplot as plt
import os
import json
import shutil
import scipy.optimize as opt
from EnergyMinimization import *

In [2]:
# root folder for data
DataFolder=os.getcwd()+'/Data/'
# Folder for the run data
#RunFolder="kc_"+"{0:0.1f}".format(kc)+"_B_"+"{0:0.1f}".format(B)+"/"
RunFolder="Scratch/"
# Name of the run
RunName=""

# Testing the Code

Make the mesh with pygmsh

In [ ]:
with pygmsh.occ.Geometry() as geom:
    geom.characteristic_length_max = 0.03
    ellipsoid = geom.add_ball([0.0, 0.0, 0.0], 1)
    InputMesh = geom.generate_mesh()
    
InputMesh.write(DataFolder+RunFolder+RunName+"InputMesh.vtk")  

In [ ]:
interiorbonds,edgebonds,boundarytris, bidxTotidx, tetras= MakeMeshData3D(InputMesh)
orientedboundarytris=OrientTriangles(InputMesh.points,boundarytris,np.array([0,0,0]))

## Testing the boundary triangle finding

when pygmsh generates a sphere, it gives the tetrahedrons and boundary triangles. We can use this to check our bondary finding is working. First, make the ball. Now, compare the lists of boundary triangles that we find to those that pygmsh finds. We need to sort the pygmsh ones, as the vertices dont always appear in ascending order

In [ ]:
np.array_equal(boundarytris,np.sort(InputMesh.cells[1].data,axis=1))

In [ ]:
boundarytris.shape

In [ ]:
InputMesh.cells[1].data.shape

This seems a good verification that our boundary list is correct

### Check total surface area

For a sphere

In [ ]:
vTotalArea3D(InputMesh.points,boundarytris)

In [ ]:
4*np.pi

## Check total volume

Ive written two functions which do this, lets check both

In [ ]:
Volume3D(InputMesh.points,orientedboundarytris,bidxTotidx).sum()

In [ ]:
Volume3D_tetras(InputMesh.points,tetras).sum()

In [ ]:
(4/3)*np.pi

## Checking the Bending Modulus Energy

 As implemented, the bending modulus approximates the continuum limit $F= \frac{\kappa_c}{2}\int dA(C_1+C_2-C_0)^2 + k_g \int dA C_1C_2$ for a closed surface, where $C_1$ etc. are the principal curvatures. According to Boal and Rao 1992, the energy of a sphere  without a spontaneous curvature is $\frac{4\pi k_{rig}}{\sqrt{3}}$, where $k_{rig}$ is the microscopic modulus.  This seems to be true for a triangulation by equilateral triangles. Lets check this:

In [ ]:
theta_0=0
kbend=1
energies, normals,na,nb= BendingEnergy(InputMesh.points,orientedboundarytris,bidxTotidx,kbend,theta_0)
energies.sum()

In [ ]:
(4/np.sqrt(3))*np.pi

## Checking the Spring Energy

In [ ]:
edgelengths= MakeBondHist(InputMesh.points,bonds)
np.mean(edgelengths)
edgelengths= MakeBondHist(OutputMesh.points,bonds)
np.mean(edgelengths)

In [ ]:
np.array_equal( np.sort(boundarytris,axis=0),np.sort(np.array(boundarytrisold),axis=0)  )
np.array_equal( np.sort(edgebondlist,axis=0),np.sort(np.array(edgebondlistold),axis=0)  )
np.array_equal( np.sort(bondlist,axis=0),np.sort(np.array(bondlistold),axis=0)  )
np.array_equal( np.sort(interiorbondlist,axis=0),np.sort(np.array(interiorbondlistold),axis=0)  )